In [ ]:
# !pip install crewai[tools]
# !pip install langchain
# !pip install qdrant-client
# !pip install langchain-groq
# !pip install sentence-transformers

## Load data

In [13]:
import pandas as pd
df = pd.read_csv("data/Adidas_final.csv", sep=';')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2625 entries, 0 to 2624
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   URL            2625 non-null   object
 1   Product Name   2625 non-null   object
 2   Product ID     2625 non-null   object
 3   Listing Price  2625 non-null   int64 
 4   Sale Price     2625 non-null   int64 
 5   Discount       2625 non-null   int64 
 6   Brand          2625 non-null   object
 7   Description    2625 non-null   object
 8   Rating         2625 non-null   int64 
 9   Reviews        2625 non-null   int64 
 10  Images         2625 non-null   object
 11  Last Visited   2625 non-null   object
dtypes: int64(5), object(7)
memory usage: 246.2+ KB


In [15]:
df.Brand.value_counts()

Brand
CORE / NEO           1111
ORIGINALS             908
SPORT PERFORMANCE     606
Name: count, dtype: int64

In [16]:
def extended_descriptin(row):
    ex_desc = f"Poduct Name: {row['Product Name'].strip()} \nDescription of product: {row['Description'].strip()}"
    return ex_desc

In [17]:
df['ExtendedDescription'] = df.apply(extended_descriptin, axis=1)

In [18]:
texts = df['ExtendedDescription'].to_list()

In [19]:
print(texts[0],"\n")

Poduct Name: Women's adidas Originals NMD_Racer Primeknit Shoes 
Description of product: Channeling the streamlined look of an '80s racer, these shoes are updated with modern features. The foot-hugging adidas Primeknit upper offers a soft, breathable feel. The Boost midsole provides responsive comfort accented with a contrast-color EVA heel plug. Embroidered details add a distinctive finish. 



## Encode Texts

In [20]:
import torch
from sentence_transformers import SentenceTransformer
device_to_use = torch.device('cuda') if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device=device_to_use)
vectors = model.encode(texts)

c:\Users\ASUS\anaconda3\envs\new\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\new\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


## Qdrant Setup

In [21]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Initialize the client

client = QdrantClient(":memory:",)
client.recreate_collection(
    collection_name="adidas_products",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

client.upload_collection(
    collection_name="adidas_products",
    ids=[i for i in range(len(texts))],
    vectors=vectors,
    parallel=4,
    max_retries=3,
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7148\2675290693.py:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [28]:
def get_matches(text):
    query_vector = model.encode(text)
    hits = client.search(
        collection_name="adidas_products",
        query_vector=query_vector,
        limit=20 # Return 20 closest points
    )
    products = list(set([df['Product Name'][i.id] for i in hits]))
    descriptions = list(set([df['Description'][i.id] for i in hits]))
    desc=""
    pdts=""
    for x, y in zip(descriptions, products):
        desc = desc + x + "\n"
        pdts = pdts + y + ', '
    return desc, pdts

In [29]:
print(get_matches("men tennis shoes"))

("Tennis-inspired style with a modern look. These men's shoes have the soft feel of a suede upper. A rubber cupsole gives comfort and ease to every stride.\nKeep your cool when it's all on the line. These lightweight tennis shoes have a breathable mesh upper to keep air circulating when the heat is on. Their foot-hugging construction and supportive chassis offer a stable feel. The durable outsole gives a firm grip with every serve and slide as you improve your game.\nServe up some style when you hit the club. The full synthetic upper provides unmatched durability. A cushioned EVA midsole keeps you comfortable from the moment you pull them on. The durable outsole stands up to every serve and slide as you improve your game.\nYou play every single day so you need a shoe that's both durable and comfortable. These adidas tennis shoes have a cushioned midsole for enhanced comfort whether you're working on your groundstroke or playing best-of-five. A durable and supportive build lets you play

## Groq API

In [30]:
from langchain_groq import ChatGroq

In [14]:
groq = ChatGroq(temperature=0.1, groq_api_key="gsk-your-groq-api-key", 
                model_name="llama3-8b-8192")

In [15]:
# system = "You are a helpful assistant."
# human = "{text}"
# prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# chain = prompt | groq
# chain.invoke({"text": "Explain the importance of low latency LLMs."})

In [16]:
from langchain_community.chat_models import ChatOllama

local_llm = "phi3"

ollama = ChatOllama(model=local_llm, temperature=0)

## CrewAI system integration

In [51]:
from crewai import Agent, Task, Crew

def agent_pipeline(query):
    desc, products = get_matches(query)
    tagline_agent = Agent(role = "Marketing and Copy writing Head",
                        goal = """Provide a catchy and creative tagline from the description that is provided JUST A TAGLINE.""",
                        backstory = """You are an excellent Marketing and Copy writing genius that doesnt explain things just gives results, who can sell any product via marketing the shit out of it.""",
                        allow_delegation = False,
                        verbose = True,
                        llm = groq)

    explainindetail_agent = Agent(role = "Senior Marketing and Copy writing Executive",
                        goal = """Provide the pros of using this product according to the product and highlight the strong points..""",
                        backstory = """You are an excellent Marketing and Copy writing genius that doesnt explain things just gives results, who can sell any product via marketing the shit out of it.""",
                        allow_delegation = False,
                        verbose = True,
                        llm = groq)

    listproducts_agent = Agent(role = "Marketing and Copy writing Analyst",
                        goal = """You'll be given a list of products, list them in presentable manner with 1 line each that's it.""",
                        backstory = """You are an excellent Marketing and Copy writing genius that doesnt explain things just gives results, who can sell any product via marketing the shit out of it.""",
                        allow_delegation = False,
                        verbose = True,
                        llm = groq)
    # manager_agent = Agent(role = "Manager",
    #                     goal = """You'll be managing all the other agents.""",
    #                     backstory = """You are an excellent man power management genius that doesnt explain things just gives results, who can maager any team via managing the shit out of it.""",
    #                     allow_delegation = False,
    #                     verbose = True,
    #                     llm = groq,
    #                     )


    task1 = Task (description=desc,
                agent = tagline_agent,
                expected_output="A tagline for the product.")

    task2 = Task (description=desc,
                agent = explainindetail_agent,
                expected_output="A description for the product.")

    task3 = Task(description=products,
                agent=listproducts_agent,
                expected_output="A numbered 1,2,3,.. list of products from the provided list.")

    results = []
    for agent, task in zip([tagline_agent, explainindetail_agent, listproducts_agent],[task1, task2, task3]):
        crew1 = Crew(agents=[agent],
                tasks=[task],
                verbose=3,
                # manager_llm=groq
            )
        result = crew1.kickoff()
        results.append(result)
    
    return {'tagline':results[0],'product_description':results[1],'products_list':results[2]}    

## Inference

In [52]:
result = agent_pipeline(query="men's casual shoes")

2024-05-29 12:52:39,725 - 9784 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Marketing and Copy writing Head
 [INFO]: == Starting Task: The adidas casual sandals for men. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsole with wet and dry traction pattern. 
Give a boost to your running or jogging sessions wearing these 8K running shoes for men. Featuring mesh and suede upper, these lace-ups are durable, light in weight and comfortable to wear. Furthermore, the grooved rubber outsole will provide better traction on the different surfaces.
These men's running shoes will get you on the road to your goals. A sandwich mesh upper offers lightweight breathability, while a seamless print overlay adds support for a stable stride. The midsole offers pillow-soft Cloudfoam cushioning that eases every stride.
The adidas casual slippers for men with a striking camouflage design. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsol

2024-05-29 12:52:41,307 - 9784 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Thought: I now can give a great answer

Final Answer: "Step Up Your Comfort Game"

> Finished chain.
 [DEBUG]: == [Marketing and Copy writing Head] Task output: "Step Up Your Comfort Game"


 [DEBUG]: == Working Agent: Senior Marketing and Copy writing Executive
 [INFO]: == Starting Task: The adidas casual sandals for men. Features stylish webbing straps, Diecut EVA midsole for stable cushioning and abrasion resistant rubber outsole with wet and dry traction pattern. 
Give a boost to your running or jogging sessions wearing these 8K running shoes for men. Featuring mesh and suede upper, these lace-ups are durable, light in weight and comfortable to wear. Furthermore, the grooved rubber outsole will provide better traction on the different surfaces.
These men's running shoes will get you on the road to your goals. A sandwich mesh upper offers lightweight breathability, while a seamless print overlay adds support for a stable stride. The midsole offers pillow-soft Cloudfoam cushioning th

2024-05-29 12:52:44,219 - 9784 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Final Answer:

**Introducing the Ultimate Comfort and Performance: adidas Casual Sandals for Men**

Step into comfort and style with the adidas Casual Sandals for men. Designed for the modern man who values both form and function, these sandals boast a sleek and stylish design that's perfect for everyday wear.

**Key Features:**

* **Stylish Webbing Straps:** Adjustable straps provide a secure and comfortable fit, while the webbing material adds a touch of sophistication to your overall look.
* **Diecut EVA Midsole:** Provides stable cushioning and support for your feet, ensuring a comfortable stride.
* **Abrasion Resistant Rubber Outsole:** With a wet and dry traction pattern, this outsole provides excellent grip on various surfaces, making it perfect for casual outings or light outdoor activities.

Whether you're running errands, meeting friends, or simply lounging around, these sandals are the perfect choice. With their lightweight and breathable design, you'll feel like you're walk

In [53]:
print(result)

{'tagline': '"Step Up Your Comfort Game"',
 'product_description': "**Introducing the Ultimate Comfort and Performance: adidas Casual Sandals for Men**\n\nStep into comfort and style with the adidas Casual Sandals for men. Designed for the modern man who values both form and function, these sandals boast a sleek and stylish design that's perfect for everyday wear.\n\n**Key Features:**\n\n* **Stylish Webbing Straps:** Adjustable straps provide a secure and comfortable fit, while the webbing material adds a touch of sophistication to your overall look.\n* **Diecut EVA Midsole:** Provides stable cushioning and support for your feet, ensuring a comfortable stride.\n* **Abrasion Resistant Rubber Outsole:** With a wet and dry traction pattern, this outsole provides excellent grip on various surfaces, making it perfect for casual outings or light outdoor activities.\n\nWhether you're running errands, meeting friends, or simply lounging around, these sandals are the perfect choice. With their 